<a href="https://colab.research.google.com/github/tonni59/Deep-Learning/blob/main/Pretrained_all_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from torchsummary import summary
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
dir(models)

['AlexNet',
 'AlexNet_Weights',
 'ConvNeXt',
 'ConvNeXt_Base_Weights',
 'ConvNeXt_Large_Weights',
 'ConvNeXt_Small_Weights',
 'ConvNeXt_Tiny_Weights',
 'DenseNet',
 'DenseNet121_Weights',
 'DenseNet161_Weights',
 'DenseNet169_Weights',
 'DenseNet201_Weights',
 'EfficientNet',
 'EfficientNet_B0_Weights',
 'EfficientNet_B1_Weights',
 'EfficientNet_B2_Weights',
 'EfficientNet_B3_Weights',
 'EfficientNet_B4_Weights',
 'EfficientNet_B5_Weights',
 'EfficientNet_B6_Weights',
 'EfficientNet_B7_Weights',
 'EfficientNet_V2_L_Weights',
 'EfficientNet_V2_M_Weights',
 'EfficientNet_V2_S_Weights',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'GoogLeNet_Weights',
 'Inception3',
 'InceptionOutputs',
 'Inception_V3_Weights',
 'MNASNet',
 'MNASNet0_5_Weights',
 'MNASNet0_75_Weights',
 'MNASNet1_0_Weights',
 'MNASNet1_3_Weights',
 'MaxVit',
 'MaxVit_T_Weights',
 'MobileNetV2',
 'MobileNetV3',
 'MobileNet_V2_Weights',
 'MobileNet_V3_Large_Weights',
 'MobileNet_V3_Small_Weights',
 'RegNet',
 'RegNet_X_16GF_Weights'

**Data load**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 3

In [ ]:
transform = transforms.Compose([transforms.Resize((227,227)),transforms.ToTensor()])

In [ ]:
train_dataset = datasets.CIFAR10(
        root='./data', train=True,
        download=True, transform=transform,
    )
test_dataset = datasets.CIFAR10(
        root='./data', train=False,
        download=True, transform=transform,
    )

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

In [ ]:
for X, y in train_loader:
  print(X.shape)
  break

torch.Size([64, 3, 227, 227])


# **AlexNet**

In [ ]:
model1 = models.alexnet(pretrained=True)
model1

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
for param in model1.parameters():
  param.requires_grad= False

In [ ]:
num_classes=10
model1.classifier[6] = nn.Linear(4096, num_classes) #layer 6 is updated
model1.classifier.add_module("7", nn.LogSoftmax(dim=1)) #activation function is added
model1

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
summary(model1,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [ ]:
loss_fn = nn.NLLLoss()
optimizer = optim.Adam(model1.parameters())
optimizer, loss_fn

(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 NLLLoss())

In [ ]:
total_step = len(train_loader)
start = time.time()
for epoch in range(num_epochs):
    epoch_start = time.time()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model1(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/3], Step [400/782], Loss: 1.3686
Epoch [2/3], Step [400/782], Loss: 0.9908
Epoch [3/3], Step [400/782], Loss: 1.0557


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model1(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


NameError: ignored

# **VGG16**

In [ ]:
model2 = models.vgg16(pretrained=True)
model2

In [ ]:
for param in model2.parameters():
  param.requires_grad= False

In [ ]:
model2.classifier = nn.Sequential(nn.Dropout(p=0.5),
                                  nn.Linear(in_features=25088, out_features=4096, bias=True),
                                  nn.ReLU(),
                                  nn.Linear(in_features=4096, out_features=1000, bias=True),
                                  nn.ReLU(),
                                  nn.Linear(in_features=1000, out_features=10, bias=True)
)
model2


In [ ]:
summary(model2,(3,224,224))

# **ResNet-50**

In [ ]:
model3 = models.resnet50(pretrained=True)
model3

In [ ]:
for param in model3.parameters():
  param.requires_grad= False

In [ ]:
in_features=model3.fc.in_features
in_features

2048

In [ ]:
model3.fc = nn.Sequential(nn.Linear(in_features, 512),
                                  nn.ReLU(),
                                  nn.Dropout(0.25),
                                  nn.Linear(512,256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, num_classes)
)
model3

In [ ]:
summary(model3,(3,224,224))

# **DenseNet-121**

In [ ]:
model4 = models.densenet121(pretrained=True)
model4

In [ ]:
for param in model4.parameters():
  param.requires_grad= False

In [ ]:
model4.classifier = nn.Sequential(nn.Linear(1024,512),
                                  nn.ReLU(),
                                  nn.Dropout(0.25),
                                  nn.Linear(512,256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, num_classes),
                                  nn.LogSoftmax(dim=1)
)
model4

# **GoogleNet**

In [ ]:
model = models.googlenet(pretrained=True)
model

In [ ]:
for param in model.parameters():
  param.requires_grad= False

In [ ]:
in_features=model.fc.in_features
in_features

1024

In [ ]:
model.fc = nn.Sequential(nn.Linear(in_features, 512),
                                  nn.ReLU(),
                                  nn.Dropout(0.25),
                                  nn.Linear(512,256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, num_classes)
)
model

# **EfficientNet-b7**

In [ ]:
model = models.efficientnet_b7(pretrained=True)
model

In [ ]:
for param in model.parameters():
  param.requires_grad= False

In [ ]:
model.classifier = nn.Sequential(nn.Linear(2560,512),
                                  nn.ReLU(),
                                  nn.Dropout(0.25),
                                  nn.Linear(512,256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, num_classes),
                                  nn.LogSoftmax(dim=1)
)
model

# **Inception v3**

In [ ]:
model = models.inception_v3(pretrained=True)
model

In [ ]:
for param in model.parameters():
  param.requires_grad= False

In [ ]:
model.fc = nn.Sequential(nn.Linear(2048,512),
                                  nn.ReLU(),
                                  nn.Dropout(0.25),
                                  nn.Linear(512,256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, num_classes),
                                  nn.LogSoftmax(dim=1)
)
model

# **MobileNet v2**

In [ ]:
model = models.mobilenet_v2(pretrained=True)
model

In [ ]:
for param in model.parameters():
  param.requires_grad= False

In [ ]:
model.fc = nn.Sequential(nn.Linear(1280,512),
                                  nn.ReLU(),
                                  nn.Dropout(0.25),
                                  nn.Linear(512,256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, num_classes),
                                  nn.LogSoftmax(dim=1)
)
model

# **Resnext50**

In [ ]:
model = models.resnext101_32x8d(pretrained=True)
model

In [ ]:
for param in model.parameters():
  param.requires_grad= False

In [ ]:
model.fc = nn.Sequential(nn.Linear(2048,512),
                                  nn.ReLU(),
                                  nn.Dropout(0.25),
                                  nn.Linear(512,256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, num_classes),
                                  nn.LogSoftmax(dim=1)
)
model

# **ShuffleNet v2**

In [ ]:
model = models.shufflenet_v2_x1_0(pretrained=True)
model

In [ ]:
in_features=model.fc.in_features
in_features

1024

In [ ]:
for param in model.parameters():
  param.requires_grad= False

In [ ]:
model.fc = nn.Linear(in_features,10)
model

# **SqueezeNet**

In [ ]:
model = models.squeezenet1_0(pretrained=True)
model

In [ ]:
for param in model.parameters():
  param.requires_grad= False

In [ ]:
model.classifier[1] = nn.Conv2d(512, num_classes, 1) #layer 6 is updated
model

# **MNASNET**

In [ ]:
model = models.mnasnet1_0(pretrained=True)
model

In [ ]:
for param in model.parameters():
  param.requires_grad= False

In [ ]:
model.classifier[1] = nn.Linear(1280, 10) #layer 6 is updated
model

# **Ensemble model (ResNet50 + ResNet18 + Custom FC)**

In [ ]:
class MyEnsemble(nn.Module):
  def __init__(self, modelA, modelB, num_classes):
    super(MyEnsemble, self).__init__()
    self.modelA= modelA
    self.modelB= modelB

    self.modelA.fc= nn.Identity()
    self.modelB.fc= nn.Identity()

    self.classifier= nn.Linear(2048+512,num_classes)

  def forward(self,x):
    x1 = self.modelA(x.clone())
    x1 = x1.view(x1.size(0),-1)
    x2 = self.modelB(x)
    x2 = x2.view(x2.size(0),-1)
    x = torch.cat((x1,x2),dim=1)
    x= self.classifier(F.softmax(x))
    return x

In [ ]:
modelA = models.resnet50(pretrained=True)
modelB = models.resnet18(pretrained=True)

In [ ]:
for param in modelA.parameters():
  param.requires_grad= False

In [ ]:
for param in modelB.parameters():
  param.requires_grad= False

In [ ]:
modelA.fc.in_features, modelB.fc.in_features

(2048, 512)

In [ ]:
model= MyEnsemble(modelA,modelB,10)
model

In [ ]:
x= torch.rand(1,3,224,224)
output= model(x)
output.shape

# **Ensemble model (ResNet50 + DenseNet121 + Custom FC)**

In [ ]:
class MyEnsemble(nn.Module):
  def __init__(self, modelA, modelB, num_classes):
    super(MyEnsemble, self).__init__()
    self.modelA= modelA
    self.modelB= modelB

    self.modelA.fc= nn.Identity()
    self.modelB.classifier= nn.Identity()

    self.classifier= nn.Sequential(
        nn.Linear(2048+1024, 512),
        nn.ReLU(),
        nn.Linear(512, 128),
        nn.ReLU(),
        nn.Linear(128, num_classes)
        )

  def forward(self,x):
    x1 = self.modelA(x.clone())
    x1 = x1.view(x1.size(0),-1)
    x2 = self.modelB(x)
    x2 = x2.view(x2.size(0),-1)
    x = torch.cat((x1,x2),dim=1)
    x= self.classifier(x)
    return x

In [ ]:
modelA = models.resnet50(pretrained=True)
modelB = models.densenet121(pretrained=True)

In [ ]:
modelB

In [ ]:
for param in modelA.parameters():
  param.requires_grad= False

for param in modelB.parameters():
  param.requires_grad= False

In [ ]:
modelA.fc.in_features, modelB.classifier.in_features

(2048, 1024)

In [ ]:
model= MyEnsemble(modelA,modelB,10)
model

In [ ]:
x= torch.rand(1,3,224,224)
output= model(x)
output.shape

torch.Size([1, 10])